<a href="https://colab.research.google.com/github/waveology/aire/blob/main/1_acceso_a_ficheros_de_datos_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# El acceso a datos almacenados en ficheros I

A veces, el acceso a los datos almacenados en ficheros puede resultar tedioso. Con frecuencia, los datos no se encuentran en el formato que desearíamos y hay invertir algo de tiempo en incorporarlos a nuestros métodos de análisis.

###1. El origen de los datos
---

Vamos a ver un ejemplo de este preproceso basado en los datos de calidad del aire que publica la Comunidad de Madrid (España) y que están libremente disponibles en [la web](https://datos.comunidad.madrid/catalogo/dataset/calidad_aire_datos_historico)

Descargamos los datos de algún año del histórico:

In [ ]:
anio = 2023

# Repositorio Github del curso donde se encuentra una copia de los datos 
# -----------------------------------------------------------------------
url = "https://raw.githubusercontent.com/waveology/aire/main/datos/comunidad"

# Descarga los datos a Colab si no se han descargado previamente
# --------------------------------------------------------------
![ ! -f {anio}.csv ] && wget {url}/{anio}.csv -O {anio}.csv


El inventario del contenido puede consultarse [aquí](https://datos.comunidad.madrid/catalogo/dataset/a770d92c-c513-4974-b1a7-2b15be1dd91f/resource/f743eacc-5e89-4591-a0fc-4caebfe22557/download/descripcion-fichero-datos-de-contaminantes.pdf)

###2. Lectura de los datos
---

Empezamos por importar las extensiones que vamos a usar:

In [ ]:
# Permite manipulación eficiente de tablas de datos
# --------------------------------------------------
import pandas as pd

A continuación, leemos los datos que están en formato CSV para generar un ***dataframe*** de Pandas. 

Especificamos que:

*   el separador de columnas es el punto y coma (;) 
*   el símbolo usado para designar decimales es la coma (,). 

NOTA: El Ayuntamiento de Madrid usa el punto (.) como separador decimal.

In [ ]:
df0 = pd.read_csv('%s.csv' % anio,  
                 sep=';', 
                 decimal=',')

Inspeccionamos la estructura del fichero:

In [ ]:
print(len(df0))        # número de filas de datos
print(df0.head(5))     # muestra las 5 primeras filas

###3. El filtrado de columnas
---

Para ahorrar memoria filtramos los datos para eliminar la información que no necesitamos.

Elegimos una estación de medida. Por ejemplo, **Guadalix de la Sierra**, a la que según el inventario le corresponde:

 *   **código**    : 28067001 (el campo **estación** solo usa los 3 últimos dígitos) 
 *   **municipio** : 67


Elegimos una estación de medida. Por ejemplo, el dióxido de nitrógeno (NO$_{2}$), a la que corresponde:

 *   **magnitud** : 8

In [ ]:
# df0 será una versión filtrada de df que solo contiene los datos seleccionados
# ------------------------------------------------------------------------------
df = df0[ (df0['magnitud']  == 8) 
       &  (df0['estacion']  == 14) 
       &  (df0['municipio'] == 65)]
       
print(len(df))
print(df.head(5))          

###4. Eliminación de columnas no necesarias
---

Una vez que hemos filtrado los datos, podemos eliminar la información que ya no necesitamos (las columnas **provincia**, **municipio**, **estación**, **punto de muestreo** y **magnitud**)

In [ ]:
df = df.drop(columns=['provincia','municipio','estacion','punto_muestreo','magnitud'])

print(df.head(5))     

###5. Pivotaje de filas y columnas
---

Nos gustaría tener en cada fila los datos correspondientes a cada tiempo. Sin embargo, la información de cada hora aparece en una columna. 

Con la función ***melt*** podemos crear un nuevo ***dataframe*** en el que las horas aparezcan en una columna y la magnitud en otra: 

In [ ]:
# df1 será una versión de df en la que las horas aparecerán en filas en lugar de columnas
# ----------------------------------------------------------------------------------------
df1 = df.melt(id_vars=['ano','mes','dia'],
                 value_vars = [ 'h%02d' % i for i in range(1,25)],
                 var_name='hora',
                 value_name='valor'
                 )
print(df1)

###6. Modificación y adaptación de valores
---

Observamos que la hora aparece en formato de texto con la letra 'h' seguida de dos dígitos. Usamos la función apply para corregirla: 

In [ ]:
# En la columna 'hora', elimina el primer carácter y convierte el resultado en numérico
# --------------------------------------------------------------------------------------
df1['hora'] = df1['hora'].apply(lambda x : int(x[1:]))

print(df1)

Repetimos la misma operación con las columnas en las que se cifra la validez de los datos:

In [ ]:
# df2 será una versión de df en la que la validez del dato aparecerán en filas en lugar de columnas
# -------------------------------------------------------------------------------------------------
df2 = df.melt(id_vars=['ano','mes','dia'],
                 value_vars = [ 'v%02d' % i for i in range(1,25)],
                 var_name='hora',
                 value_name='flag'
                 )
df2['hora'] = df2['hora'].apply(lambda x : int(x[1:]))
print(df2)

###7. Fusión de tablas
---

Hemos creado dos ***dataframes*** (df1 y df2) que contienen los registros de contaminante y la validez de los datos respectivamente. 

A continuación los fusionamos con la función ***merge***:

In [ ]:
print(df1)
print(df2)
df = df1.merge(df2)
print(df)

###8. Eliminación de datos no válidos
---

Seleccionamos solo los datos válidos. Después eliminamos la columna 'flag', que ya no será necesaria. 

In [ ]:
n_antes = len(df)

df = df[df['flag'] == 'V'].drop(columns='flag')
print(df)

n_despues = len(df)
print('Eliminados %d datos malos' % (n_antes - n_despues))

###9. El tiempo
---

Ahora nos vendría muy bien que el tiempo estuviera en una única columna que incluya la fecha y la hora (***datetime***):

In [ ]:
# Creamos una columna de tiempo indicando qué columnas tienen el año, el mes, el día y la hora
# ---------------------------------------------------------------------------------------------
df['fecha'] = pd.to_datetime({'year':df.ano,'month':df.mes,'day':df.dia,'hour':df.hora})
print(df)

Ya no necesitamos las columnas de año, mes, día y hora:

In [ ]:
df = df.drop(columns=['ano','mes','dia','hora'])
print(df)

###10. Reordenación de columnas si es necesario
---

Por lo general no es preciso reordenar las columnas pero podemos hacerlo así:

In [ ]:

df = df[['fecha','valor']]
print(df)

###11. Resultado final
---

Ahora nuestros datos están dispuestos en un formato que simplifica muchas tareas de análisis:

In [ ]:
df.plot(x='fecha',y='valor')

Podemos crear una versión más detallada especificando parámetros en la función plot:

In [ ]:
df.plot(x='fecha',y='valor',
        fontsize=12,
        figsize=(15,10),
        marker='o',
        ms=5,
        lw=1,
        grid=True,
        legend=False,
        title = 'Concentración de NO$_{2}$ ($\mu$g/m$^{3}$) medida en la estación de Guadalix de la Sierra'
        )